In [39]:
import tensorflow as tf
import os

# Caminho
model_path = "corr_model_optimized.tflite"
if os.path.exists(model_path):
    os.remove(model_path)

class CorrelationModel(tf.Module):
    @tf.function(input_signature=[tf.TensorSpec(shape=[3, None], dtype=tf.float32)])
    def __call__(self, inputs):
        x = inputs[0]
        y = inputs[1]
        z = inputs[2]

        def corr(a, b):
            a_mean = tf.reduce_mean(a)
            b_mean = tf.reduce_mean(b)
            a_centered = a - a_mean
            b_centered = b - b_mean

            numerator = tf.reduce_sum(a_centered * b_centered)
            denom = tf.sqrt(tf.reduce_sum(a_centered ** 2) * tf.reduce_sum(b_centered ** 2))
            return numerator / denom

        c_xy = corr(x, y)
        c_xz = corr(x, z)
        c_yz = corr(y, z)

        return tf.stack([c_xy, c_xz, c_yz])

# Exportação
model = CorrelationModel()
concrete_fn = model.__call__.get_concrete_function()
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_fn], model)

converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()

with open(model_path, "wb") as f:
    f.write(tflite_model)

print("✅ Modelo otimizado salvo como:", model_path)

INFO:tensorflow:Assets written to: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp9nskphxd/assets


INFO:tensorflow:Assets written to: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp9nskphxd/assets


✅ Modelo otimizado salvo como: corr_model_optimized.tflite


W0000 00:00:1750714321.695085 22886317 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1750714321.695096 22886317 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-06-23 18:32:01.695206: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp9nskphxd
2025-06-23 18:32:01.695395: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-06-23 18:32:01.695399: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp9nskphxd
2025-06-23 18:32:01.696734: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-06-23 18:32:01.701231: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp9nskphxd
2025-06-23 18:32:01.704539: I tensorflow/cc/saved_model/loader.c

In [41]:
import numpy as np
import tensorflow as tf
import time
import os
import psutil

# --- Caminho do modelo ---
model_path = "corr_model_optimized.tflite"

# --- Função de referência ---
def cross_axis_corr_py(data):
    x, y, z = data
    corr_xy = np.corrcoef(x, y)[0, 1]
    corr_xz = np.corrcoef(x, z)[0, 1]
    corr_yz = np.corrcoef(y, z)[0, 1]
    return np.array([corr_xy, corr_xz, corr_yz], dtype=np.float32)

# --- Geração dos dados ---
N = 10_000
x = np.random.rand(N).astype(np.float32)
y = np.random.rand(N).astype(np.float32)
z = np.random.rand(N).astype(np.float32)
sensor_data = np.stack([x, y, z], axis=0)  # shape [3, N]

# --- Cálculo CPU ---
start_cpu = time.time()
corr_ref = cross_axis_corr_py(sensor_data)
end_cpu = time.time()
cpu_time_ms = (end_cpu - start_cpu) * 1000

# --- TFLite ---
start_load = time.time()
interpreter = tf.lite.Interpreter(model_path=model_path)
end_load = time.time()
load_time_ms = (end_load - start_load) * 1000

interpreter.resize_tensor_input(0, [3, N])
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

start_tflite = time.time()
interpreter.set_tensor(input_details[0]['index'], sensor_data.astype(np.float32))
interpreter.invoke()
corr_out = interpreter.get_tensor(output_details[0]['index'])[0]
end_tflite = time.time()
tflite_time_ms = (end_tflite - start_tflite) * 1000

# --- Resultados ---
erro_abs = np.abs(corr_ref - corr_out)
reduction_pct = ((cpu_time_ms - tflite_time_ms) / cpu_time_ms) * 100
model_size_kb = os.path.getsize(model_path) / 1024
mem_used_mb = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024

print("📊 Resultado:")
print("Referência (Python):", corr_ref)
print("Modelo TFLite      :", corr_out)
print("Erro absoluto      :", erro_abs)

print(f"\n⏱️ Tempo CPU:     {cpu_time_ms:.2f} ms")
print(f"📦 Tempo carga:    {load_time_ms:.2f} ms")
print(f"⚡ Tempo TFLite:   {tflite_time_ms:.2f} ms")
print(f"📉 Redução tempo : {reduction_pct:.2f}%")
print(f"💾 Tamanho modelo : {model_size_kb:.2f} KB")
print(f"🧠 Memória processo: {mem_used_mb:.2f} MB")

📊 Resultado:
Referência (Python): [-0.00250173 -0.01864572 -0.00463506]
Modelo TFLite      : -0.0025017257
Erro absoluto      : [4.6566129e-09 1.6143994e-02 2.1333378e-03]

⏱️ Tempo CPU:     1.03 ms
📦 Tempo carga:    0.53 ms
⚡ Tempo TFLite:   0.18 ms
📉 Redução tempo : 82.85%
💾 Tamanho modelo : 4.95 KB
🧠 Memória processo: 354.42 MB


In [43]:
import tensorflow as tf
import os

# Caminho do modelo
model_path = "correlation_model_windowed.tflite"
if os.path.exists(model_path):
    os.remove(model_path)

class WindowedCrossAxisCorrelationModel(tf.Module):
    @tf.function(input_signature=[tf.TensorSpec(shape=[4, None], dtype=tf.int32)])
    def __call__(self, inputs):
        timestamps = tf.cast(inputs[0], tf.float32)
        x = tf.cast(inputs[1], tf.float32)
        y = tf.cast(inputs[2], tf.float32)
        z = tf.cast(inputs[3], tf.float32)

        # IDs de janelas (a cada 5000 ms)
        win = tf.cast(tf.math.floor(timestamps / 5000.0), tf.int32)
        win = win - tf.reduce_min(win)  # começa do zero
        num_windows = tf.reduce_max(win) + 1

        def correlation(a, b):
            a_mean = tf.math.unsorted_segment_mean(a, win, num_windows)
            b_mean = tf.math.unsorted_segment_mean(b, win, num_windows)

            a_centered = a - tf.gather(a_mean, win)
            b_centered = b - tf.gather(b_mean, win)

            cov = tf.math.unsorted_segment_mean(a_centered * b_centered, win, num_windows)
            std_a = tf.sqrt(tf.math.unsorted_segment_mean(a_centered ** 2, win, num_windows))
            std_b = tf.sqrt(tf.math.unsorted_segment_mean(b_centered ** 2, win, num_windows))

            denom = std_a * std_b + 1e-8  # evitar divisão por zero
            return cov / denom

        corr_xy = correlation(x, y)
        corr_xz = correlation(x, z)
        corr_yz = correlation(y, z)

        # Média global das correlações por janela
        return tf.stack([
            tf.reduce_mean(corr_xy),
            tf.reduce_mean(corr_xz),
            tf.reduce_mean(corr_yz)
        ])

# Exporta
model = WindowedCrossAxisCorrelationModel()
concrete_fn = model.__call__.get_concrete_function()
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_fn], model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()

with open(model_path, "wb") as f:
    f.write(tflite_model)

print("✅ Modelo TFLite otimizado com janelas salvo:", model_path)

INFO:tensorflow:Assets written to: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp11t_r3uz/assets


INFO:tensorflow:Assets written to: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp11t_r3uz/assets
W0000 00:00:1750714333.217793 22886317 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1750714333.217802 22886317 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-06-23 18:32:13.217895: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp11t_r3uz
2025-06-23 18:32:13.218425: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-06-23 18:32:13.218430: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmp11t_r3uz
2025-06-23 18:32:13.224674: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-06-23 18:32:13.236718: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/qv/p5h208

✅ Modelo TFLite otimizado com janelas salvo: correlation_model_windowed.tflite


In [51]:
import numpy as np
import tensorflow as tf
import time
import os

# --- Função de referência em Python puro ---
def windowed_correlation_py(data, window_size_ms=5000):
    timestamps = data[0].astype(np.float32)
    x = data[1].astype(np.float32)
    y = data[2].astype(np.float32)
    z = data[3].astype(np.float32)

    window_ids = np.floor(timestamps / window_size_ms).astype(np.int32)
    unique_windows = np.unique(window_ids)

    correlations_xy, correlations_xz, correlations_yz = [], [], []

    for wid in unique_windows:
        idx = np.where(window_ids == wid)[0]
        if len(idx) > 1:
            xw, yw, zw = x[idx], y[idx], z[idx]
            correlations_xy.append(np.corrcoef(xw, yw)[0, 1])
            correlations_xz.append(np.corrcoef(xw, zw)[0, 1])
            correlations_yz.append(np.corrcoef(yw, zw)[0, 1])

    return np.array([
        np.mean(correlations_xy),
        np.mean(correlations_xz),
        np.mean(correlations_yz)
    ], dtype=np.float32)

# --- Geração de dados simulados ---
N = 10000
timestamps = (np.arange(N) * 20).astype(np.int32)
x = (np.random.rand(N) * 10).astype(np.float32)
y = (np.random.rand(N) * 10).astype(np.float32)
z = (np.random.rand(N) * 10).astype(np.float32)
sensor_data = np.stack([timestamps, x, y, z], axis=0)  # shape [4, N]

# --- Referência CPU ---
start_cpu = time.time()
corr_ref = windowed_correlation_py(sensor_data)
cpu_time = (time.time() - start_cpu) * 1000

# --- TFLite ---
model_path = "correlation_model_windowed.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.resize_tensor_input(0, [4, N])
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

start_infer = time.time()
interpreter.set_tensor(input_details[0]['index'], sensor_data.astype(np.int32))
interpreter.invoke()
corr_out = interpreter.get_tensor(output_details[0]['index'])[0]
tflite_time = (time.time() - start_infer) * 1000

# --- Resultados ---
print("Referência (Python):", corr_ref)
print("Modelo TFLite      :", corr_out)
print("Erro absoluto      :", np.abs(corr_ref - corr_out))
print(f"\n⏱️ Tempo CPU:     {cpu_time:.2f} ms")
print(f"⚡ Tempo TFLite:   {tflite_time:.2f} ms")
print(f"📉 Redução tempo : {(cpu_time - tflite_time) / cpu_time * 100:.2f}%")
print(f"📦 Tamanho modelo : {os.path.getsize(model_path) / 1024:.2f} KB")

Referência (Python): [-0.00687676  0.01214411 -0.00642308]
Modelo TFLite      : -0.0051486674
Erro absoluto      : [0.00172809 0.01729278 0.00127442]

⏱️ Tempo CPU:     7.31 ms
⚡ Tempo TFLite:   0.94 ms
📉 Redução tempo : 87.14%
📦 Tamanho modelo : 10.74 KB
